## Loading and preprocessing ISTAT data

Processing pipeline to collect Istat cpa 2011 data for selected cities from downloaded data dump.

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from os import path, listdir

In [2]:
## TODO: find way to put this into some global settings
import os
import sys
# go up two levels in path
nb_dir = os.path.dirname(os.path.dirname(os.getcwd()))
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

from references import common_cfg
sezNameCol = common_cfg.sezioneColName

In [3]:
## Loading ISTAT data
fileList = common_cfg.get_istat_filelist()
dfList = []
dataDict = {}
for filename in fileList:
    regionData = pd.read_csv(
        path.join(common_cfg.cpaPath, filename), sep=';', encoding='latin').set_index('SEZ2011')
    # extract the councils we are intersted in
    for city in common_cfg.cityList:
        cityData = regionData[regionData.COMUNE==city]
        if cityData.size>0:
            dataDict[city] = cityData
            
# export as csv to final folder for every city
for city, cityData in dataDict.items():
    pass
    cityData.to_csv('../../data/processed/istat_cpa_2011/' + city + '_cpa_2011.csv', sep=';')

### Process ISTAT data for Milano to assign a quartiere label to each sezione

In [ ]:
# Join sez geofile for Milano - bespoke processing
shapeDataMilano = gpd.read_file('../../data/raw/Milano_specific/Milano_sezioniShapefile')

# Convert to epsg 4326
shapeDataMilano = shapeDataMilano.to_crs({'init':'epsg:4326'})

# These aren't exactly equal to 1, need to check mismatches in joining
print(shapeDataMilano[sezNameCol].isin(dataDict['Milano'].index).mean())
print(dataDict['Milano'].index.isin(shapeDataMilano[sezNameCol]).mean())

joinedData = pd.merge(shapeDataMilano, dataDict['Milano'], how='inner',
                      right_index=True, left_on=common_cfg.sezioneColName)
# Cast as int
joinedData[sezNameCol] = joinedData[sezNameCol].astype(float)

In [ ]:
# collect quartiere label
quartiereLabels = pd.read_csv('../../data/raw/Milano_specific/Milano_sezToQuartieri.csv')

# fix typo
quartiereLabels.NIL.replace(to_replace='MAGENTA - S.VITTORE', value='MAGENTA - S. VITTORE', inplace=True)

# rename to match common config
quartiereLabels.rename(
    columns={'NIL': common_cfg.quartiereDescColName}, inplace=True)

# load name-id table for quartieri
quartieriData = gpd.read_file('../../data/raw/Milano_specific/Milano_quartieri.geojson')

# join
quartiereLabels = quartiereLabels.join(
    quartieriData[[common_cfg.quartiereDescColName,
                   common_cfg.IdQuartiereColName]].set_index(
        common_cfg.quartiereDescColName), on=common_cfg.quartiereDescColName)
quartiereLabels = quartiereLabels.set_index(sezNameCol)

assert not any(quartiereLabels[common_cfg.IdQuartiereColName].isnull()), 'Typos in NIL field'

In [ ]:
joinedData = joinedData.join(quartiereLabels, on=sezNameCol)
joinedData = joinedData[~joinedData[common_cfg.IdQuartiereColName].isnull()]
joinedData[common_cfg.IdQuartiereColName] = joinedData[common_cfg.IdQuartiereColName].astype(int)

In [ ]:
outMilanoFilename = '../../data/processed/Milano_sezioni.geojson'
joinedData.to_file(outMilanoFilename, driver='GeoJSON')

In [ ]:
# check loading
print(gpd.read_file(outMilanoFilename))

### Process ISTAT data for Turin to assign a circoscrizione label to each sezione

Data for "Circoscrizioni", "Quartieri" and "Zone statistiche" are already stored in DAF under "comune_torino_csi". 

They are private, in the future we can get data using API.

In [36]:
#city
selected_city = "Torino"
custom_turin_label = "NCIRCO"
#prefix for ISTAT SEZ2011
prefix_torino = 12720000000

# load name-id table for circoscrizioni, quartieri and sezioni censimento
circoscrizioniData = gpd.read_file('../../data/raw/Torino_specific/circoscrizioni_wgs84.geojson')
sezioniData = gpd.read_file ('../../data/raw/Torino_specific/sezioni_censimento_wgs84.geojson')
quartieriData = gpd.read_file('../../data/raw/Torino_specific/ex_quartieri_wsg84.geojson')

In [37]:
#quartieriData.plot()
#circoscrizioniData.plot()
#sezioniData.plot()

In [38]:
#cast sezcens to int
sezioniData.SEZCENS = sezioniData.SEZCENS.astype(int)
sezioniData.NCIRCO = sezioniData.NCIRCO.astype(int)

#Join with circoscrizioniData to get the name, drop circoscrizione geometry we don't need them at this level
circoscrizioniData.drop('geometry', axis=1, inplace=True)
sezioniData= pd.merge(sezioniData,circoscrizioniData, on=custom_turin_label)
#rename NCIRCO field to IDquartiere
sezioniData.rename(columns={custom_turin_label : common_cfg.IdQuartiereColName}, inplace=True)
#add prefix to Join ISTAT
sezioniData.SEZCENS = sezioniData.SEZCENS + prefix_torino
#set index
sezioniData.set_index(sezioniData.SEZCENS, inplace=True)
#Join ISTAT DATA + Circoscrizione name and circoscrizione id
joinedData = pd.merge(sezioniData, dataDict[selected_city], 
                      how='inner',
                      right_index=True, 
                      left_index=True)

joinedData.rename(columns={'SEZCENS' : common_cfg.sezioneColName}, inplace=True)


In [39]:
#Save Final geojson
outFilename = '../../data/processed/'+selected_city+'_sezioni.geojson'
joinedData.to_file(outFilename, driver='GeoJSON')

In [ ]:
#joinedData.plot()

### Check Population and Section data Istat 2011 and Turin 2017

In [ ]:
# ISTAT Turin
istatData = dataDict[selected_city] #Population from ISTAT (2011)
popTotIstat = istatData.P1.sum()
popMaxTotIstat = istatData.P1.max()
popMinTotIstat = istatData.P1.min()
sezTotIstat = istatData.NSEZ.count()

print ("Popolazione 2011 da ISTAT: ", popTotIstat, "Sezioni:" ,sezTotIstat, "Max sezione:", popMaxTotIstat, "Mix sezione:", popMinTotIstat, )

In [ ]:
#Population from Turin (2017)
popTotSezioni= sezioniData.NRESID.sum()
popMaxSezioni= sezioniData.NRESID.max()
popMinSezioni= sezioniData.NRESID.min()
sezTotSezioni = sezioniData.SEZCENS.count()
print ("Popolazione 2017 da comune: ", popTotSezioni ,"Sezioni:" ,sezTotSezioni, "Max sezione:", popMaxSezioni, "Min sezione:", popMinSezioni, )
print ("Diff pop:", popTotSezioni-popTotIstat, " -- Diff sezioni:",sezTotSezioni- sezTotIstat)

In [ ]:
#Check empty sezioni 
print( "Istat 2011: ",len(istatData.loc[istatData.P1 == 0, :]))